In [12]:
import pymysql
import copy
import json

In [38]:
def upgrade(state):
    state = copy.deepcopy(state)
    
    if "cuiAssignment" not in state:
        print(" • Create cuiAssignment")
        state['cuiAssignment'] = {}
        cuis = [concept['cui'] for concept in state['mapping']['concepts']]
        for concept in state['indexing']['concepts']:
            include = concept['cui'] in cuis
            state['cuiAssignment'][concept['cui']] = \
                "include" if include else "exclude"
                
    if 'codingSystems' not in state:
        print(" • Move codingSystems")
        assert state['mapping']['codingSystems']
        state['codingSystems'] = state['mapping']['codingSystems']
        del state['mapping']['codingSystems']
        
    if 'semanticTypes' in state['mapping']:
        print(" • Remove semanticTypes")
        del state['mapping']['semanticTypes']
    
    if any('codingSystem' not in sourceConcept
           for concept in state['mapping']['concepts']
           for sourceConcept in concept['sourceConcepts']):
        print(" • Upgrade codingSystem fields")
        for concept in state['mapping']['concepts']:
            for sourceConcept in concept['sourceConcepts']:
                assert 'vocabulary' in sourceConcept
                sourceConcept['codingSystem'] = sourceConcept['vocabulary']
                del sourceConcept['vocabulary']

    return state

In [46]:
connection = pymysql.connect("127.0.0.1", "root", "root", "code-mapper-testing", cursorclass=pymysql.cursors.DictCursor)

In [56]:
query = """
    SELECT id, name, state FROM `case_definitions`
"""
update = """
    UPDATE `case_definitions` SET `state` = %s WHERE `id` = %s
"""
with connection.cursor() as cursor:
    cursor.execute(query)
    for case_definition in cursor.fetchall():
        print(case_definition['name'])
        state = json.loads(case_definition['state'])
        new_state = json.dumps(upgrade(state))
        cursor.execute(update, (new_state, case_definition['id']))
        connection.commit()

Pertussis BB
test BB
Infectious disease KG
test rh
Dementia
thrombocytopenia BC
thrombocytopenia PLAIN
IHD_pmc
pertussis PLAIN
pertussis ECDC
anaphylaxis PLAIN
Intussusception_pmc
Thromboembolic events_pmc
Rhabdomyolysis_pmc
test again)IHD_pmc
test2
anaphylaxis BC
anaphylaxis US-IAID
gbs PLAIN
gbs BC
uiae
test BBx
